In [1]:
from mpmath import *
import sys
mp.dps = 18; mp.pretty = True

def erfcApr(x):
  z = abs(x);
  t = 1 / (1 + z / 2);
  r = t * exp(-z * z - mpf(1.26551223) + t * (mpf(1.00002368) +
            t * (mpf(0.37409196) + t * (mpf(0.09678418) + t * (mpf(-0.18628806) +
            t * (mpf(0.27886807) + t * (mpf(-1.13520398) + t * (mpf(1.48851587) +
            t * (mpf(-0.82215223) + t * mpf(0.17087277))))))))))
  return r if x >= 0 else 2 - r;

In [2]:
def findMinN(start, end, maxErr):
  n = 1
  apr_poly,apr_err = chebyfit(erfcApr, [start, end], n, error=True)
  while(apr_err>maxErr):
    n += 1
    apr_poly,apr_err = chebyfit(erfcApr, [start, end], n, error=True)
  return apr_poly,apr_err,n

In [3]:
def findIntervals(start, end, maxErr, maxPolyDeg, prec=mpf("0.01")):
  maxREnd = start
  l = start
  r = end
  while(l<=r-prec):
    mid = (l+r)/2
    poly,err,cheb_deg = findMinN(start,mid, maxErr)
    if (cheb_deg<=maxPolyDeg):
      maxREnd = mid
      l = mid
    else:
      r = mid
  if(maxREnd>=end-prec):
    # We covered the whole interval
    return [(start, end, (err, poly))]
  else:
    return [(start, maxREnd, (err, poly))] + findIntervals(maxREnd, end, maxErr, maxPolyDeg, prec)

In [4]:
maxEr = mpf("1e-8")
a = mpf("0")
b = mpf("6.24")

for max_poly_deg in range(26,2,-1):
  intervals = findIntervals(a,b,maxEr,max_poly_deg,prec=mpf("0.0000001"))
  print(f"Max degree: {max_poly_deg}, intervals number: {len(intervals)}, intervals: {intervals}")

Max degree: 26, intervals number: 1, intervals: [(0.0, 6.24, (7.07136102279702359e-9, [2.35303852350415968e-13, -1.79650021036665447e-11, 6.41316135572316863e-10, -1.42094613449361125e-8, 2.18716355498748383e-7, -2.47993940912161502e-6, 0.0000214278231519267562, -0.000143941783670529834, 0.000760069616635675832, -0.00316928021138466261, 0.0104342602259019238, -0.0270564094427213377, 0.0552430572620798423, -0.0897983138836376996, 0.119683046624680173, -0.132871235648176518, 0.110209110981269878, -0.0499497698900735707, 0.0426551089891773893, -0.131126798295549522, 0.00571673927428194877, 0.374941210664231681, 0.000139075191135467865, -1.12838589774345848, 1.00000002794982858]))]


KeyboardInterrupt: 

In [5]:
a = mpf("0")
b = mpf("6.24")
maxEr = mpf("1e-9")
max_poly_deg = 5
intervals = findIntervals(a,b,maxEr,max_poly_deg,prec=mpf("0.0000001"))
ends = []
print(f"Max degree: {max_poly_deg}, intervals number: {len(intervals)}")
for i in range(len(intervals)):
  start, end, (err, poly) = intervals[i]
  print('old',poly)
  poly = [int(mpf('1e18')*poly[j]) for j in range(len(poly))]
  print('new',poly)
  int_end = int(mpf('1e18')*end)
  intervals[i] = (start, int_end, (err, poly))
  ends.append(int_end)

  print(f"Interval: [{start}, {end}]. Error: {err}. Poly: \n", \
   *[f"poly{i}[{j}] = {poly[j]};\n" for j in range(len(poly))])

Max degree: 5, intervals number: 29
old [-0.124620041230776014, 0.00594883826956300222, 0.374802697318461983, 0.000151951361675853106, -1.12838630244934578, 1.00000002999258644]
new [-124620041230776013, 5948838269563002, 374802697318461983, 151951361675853, -1128386302449345780, 1000000029992586435]
Interval: [0.0, 0.0835052776336669922]. Error: 7.36106893691432508e-12. Poly: 
 poly0[0] = -124620041230776013;
 poly0[1] = 5948838269563002;
 poly0[2] = 374802697318461983;
 poly0[3] = 151951361675853;
 poly0[4] = -1128386302449345780;
 poly0[5] = 1000000029992586435;

old [-0.0689735423028576442, 0.392951190781528111, -0.00203891235999828982, -1.12825526344796592, 0.999996933043385613]
new [-68973542302857644, 392951190781528110, -2038912359998289, -1128255263447965918, 999996933043385612]
Interval: [0.0835052776336669922, 0.169659591110680736]. Error: 9.99996541155601201e-10. Poly: 
 poly1[0] = -68973542302857644;
 poly1[1] = 392951190781528110;
 poly1[2] = -2038912359998289;
 poly1[3] 

In [6]:
def ChebErfc(x):
  assert(mpf("0")<=x<=mpf("6.24"))
  for i in intervals:
    start, end, (err, poly) = i
    if(start<=x<=end):
      # erfc(x) can be approximated by this poly
      return polyval(poly, x)

In [7]:
def evm_hex(x):
  #return x
  if x>=0:
    return hex(x)
  return hex(int("0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff", 16)+x+1)

In [9]:
for i in range(len(ends)-1):
  assert ends[i]<ends[i+1]

def polyvalOpcodes(poly):
  n = len(poly)

  wadN = n-1
  s = "[WAD] "*wadN+f"{evm_hex(poly[0])} "
  for i in range(len(poly)-1):
    s = s + f"dup{2+wadN-i} mul sdiv {evm_hex(poly[i+1])} add "
  s = s + "polyval_end jump"
  return s

def getPolyByIntervalEnd(intervals, v: int):
  for i in range(len(intervals)):
    start, end, (err, poly) = intervals[i]
    if end==v:
      return poly

def printBisectTree(a, start, end, depth, was_to_left, hist, intervals):
  if(was_to_left):
    print('pivot'+hist+':')

  if start+2>=end:
    #print(f"interval end value = {a[end-1]}")
    print(polyvalOpcodes(getPolyByIntervalEnd(intervals, a[end-1])))
    return

  mid = (start+end)//2

  print(evm_hex(a[mid]), 'dup2 lt pivot'+hist+'0', 'jumpi')

  printBisectTree(a,mid,end,depth+1, False, hist+'1', intervals)
  printBisectTree(a, start, mid+1, depth+1, True, hist+'0', intervals)

In [10]:
print("Huff binary search & poly evaluation code:")
printBisectTree([0]+ends, 0, len(ends)+1, 0, False, '', intervals)

Huff binary search & poly evaluation code:
0x15276123ba5a365c dup2 lt pivot0 jumpi
0x22e2f8dd9b2090f1 dup2 lt pivot10 jumpi
0x2e0de3be1f98084c dup2 lt pivot110 jumpi
0x3b2b561061d447bf dup2 lt pivot1110 jumpi
0x1b166 polyval_end jump
pivot1110:
0x32e0bb973dc067bf dup2 lt pivot11100 jumpi
[WAD] [WAD] [WAD] [WAD] [WAD] 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffb4f7b83d7fc dup7 mul sdiv 0x6080f6e5eec1 add dup6 mul sdiv 0xfffffffffffffffffffffffffffffffffffffffffffffffffffce52c28a11d40 add dup5 mul sdiv 0xccb4ee33d254e add dup4 mul sdiv 0xffffffffffffffffffffffffffffffffffffffffffffffffffe59ec2dae2a8e3 add dup3 mul sdiv 0x15c61adf723f84 add polyval_end jump
pivot11100:
[WAD] [WAD] [WAD] [WAD] 0x43531febefb4 dup6 mul sdiv 0xfffffffffffffffffffffffffffffffffffffffffffffffffffc281c314ae6c7 add dup5 mul sdiv 0x1517938d3eea9c add dup4 mul sdiv 0xffffffffffffffffffffffffffffffffffffffffffffffffffcc79d7a6ceea5b add dup3 mul sdiv 0x2f46c81da53c92 add polyval_end jump
pivot110:
0x279af